In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')

import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.io.img_tiles as cimgt

import AISMapPlot as aismap

In [2]:
ais_learners = []
sectors = [0, 1, 2]
sector_borders = [2.2, 3.20, 3.94]
positions = pd.DataFrame([[np.nan, 2.2], [np.nan, 3.2], [np.nan, 3.94]],
                         columns = ['Latitude', 'Longitude'])

for s in sectors:
    ais_learners.append(pd.read_pickle('FEL-ROT_learners_01min_s%d.pkl' % (s)))

In [3]:
positions

,Latitude,Longitude
0,NaN,2.20
1,NaN,3.20
2,NaN,3.94


In [4]:
ais_learners[0].head()

,TripID,time,Length,Breadth,Draught,Latitude,Longitude,SOG,COG,TH,...,hour_of_day,weekday,day_of_month,month,sector,mins_to_sector_leave,sector_leave_lat,sector_leave_lon,sector_leave_sog,sector_leave_cog
336698,1000064,2016-03-14 23:40:00,82.0,11.0,3.04,52.03,1.69,8.1,88.0,76.0,...,23,0,14,3,0,136.615385,52.0,2.2,9.130769,99.1
336699,1000064,2016-03-14 23:41:00,82.0,11.0,3.04,52.03,1.70,8.2,88.1,77.0,...,23,0,14,3,0,135.615385,52.0,2.2,9.130769,99.1
336700,1000064,2016-03-14 23:42:00,82.0,11.0,3.04,52.03,1.70,8.3,90.1,78.0,...,23,0,14,3,0,134.615385,52.0,2.2,9.130769,99.1
336701,1000064,2016-03-14 23:43:00,82.0,11.0,3.04,52.03,1.70,8.5,88.7,76.0,...,23,0,14,3,0,133.615385,52.0,2.2,9.130769,99.1
336533,1000064,2016-03-14 23:44:00,82.0,11.0,3.04,52.03,1.71,8.2,87.9,77.0,...,23,0,14,3,0,132.615385,52.0,2.2,9.130769,99.1


In [5]:
X = []
y = []
for s in sectors:
    X.append(ais_learners[s][['Latitude', 'Longitude', 'COG', 'SOG']])
    y.append(ais_learners[s][['sector_leave_lat', 'sector_leave_lon', 'mins_to_sector_leave', 'sector_leave_cog', 'sector_leave_sog']])

In [6]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score

cv_splits = []
for s in sectors:
    cv_splits.append(list(GroupKFold(n_splits = 10).split(X[s], y[s], groups = ais_learners[s]['TripID'])))

In [7]:
# Retrieve ONE simple train-test split (for each sector)
train_indices = []
test_indices = []
for s in sectors:
    a, b = cv_splits[s][0]
    train_indices.append(a)
    test_indices.append(b)

X_train = []
X_test = []
for s in sectors:
    X_train.append(X[s].iloc[train_indices[s]])
    X_test.append(X[s].iloc[test_indices[s]])

y_train = []
y_test = []
for s in sectors:
    y_train.append(y[s].iloc[train_indices[s]])
    y_test.append(y[s].iloc[test_indices[s]])

In [8]:
from sklearn.neighbors import KNeighborsRegressor

for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 20)
    scores = cross_val_score(knn_pos, X[s], y[s], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print(scores.mean())

-4.124034658807982
-4.318491586996478
-3.9805115785377074


In [ ]:
#3.40111799045

from sklearn.neighbors import KNeighborsRegressor

for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 7, weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_pos, X[s], y[s], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print(scores.mean())

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_pos, X[s], y[s], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print(scores.mean())

In [9]:
knn_pos_list = []
for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    knn_pos.fit(X_train[s], y_train[s]['sector_leave_lat'])
    knn_pos_list.append(knn_pos)

In [44]:
knn_cog_list = []
for s in sectors:
    knn_cog = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    knn_cog.fit(X_train[s], y_train[s]['sector_leave_cog'])
    knn_cog_list.append(knn_cog)

In [11]:
knn_sog_list = []
for s in sectors:
    knn_sog = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    knn_sog.fit(X_train[s], y_train[s]['sector_leave_sog'])
    knn_sog_list.append(knn_sog)

In [13]:
knn_time_list = []
for s in sectors:
    knn_time = KNeighborsRegressor(n_neighbors = 5, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    knn_time.fit(X_train[s], y_train[s]['mins_to_sector_leave'])
    knn_time_list.append(knn_time)

In [12]:
knn_pos_list[1].predict([[51.88, 2.88, 84.0, 15.8]])

array([51.92718176])

In [ ]:
# 51.99 4.07
lats = [52.03]
lons = [1.70]
for s in sectors:
    lat = knn_pos_list[s].predict([[lats[-1], lons[-1]]])[0]
    lon = sector_borders[s]
    lats.append(lat)
    lons.append(lon)
print(lats)
print(lons)

In [ ]:
aismap.plot_to_map(size = (13, 13), longitude = pd.Series(lons), latitude = pd.Series(lats))

In [ ]:
for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 5, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_pos, X[s], y[s]['sector_leave_lat'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

In [13]:
for s in sectors:
    knn_time = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_time, X[s], y[s]['mins_to_sector_leave'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

0 -> -11.253571
1 -> -13.020298
2 -> -10.696820


In [16]:
for s in sectors:
    knn_time = KNeighborsRegressor(n_neighbors = 5, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_time, X[s], y[s]['mins_to_sector_leave'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

0 -> -11.309401
1 -> -12.692429
2 -> -10.761773


In [17]:
for s in sectors:
    knn_pos = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_pos, X[s], y[s]['sector_leave_lat'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

0 -> -0.025594
1 -> -0.058227
2 -> -0.003658


In [14]:
for s in sectors:
    knn_sog = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_sog, X[s], y[s]['sector_leave_sog'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

0 -> -1.171854
1 -> -0.978377
2 -> -1.283975


In [15]:
for s in sectors:
    knn_cog = KNeighborsRegressor(n_neighbors = 10, algorithm = 'kd_tree', weights = 'distance', leaf_size = 30)
    scores = cross_val_score(knn_cog, X[s], y[s]['sector_leave_cog'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))

0 -> -5.311869
1 -> -3.107754
2 -> -4.248056


In [ ]:
knn_time_list[1].predict([[51.93, 1.3]])[0]

In [28]:
from sklearn.ensemble import RandomForestRegressor
for s in sectors:
    rfr_time = RandomForestRegressor(n_estimators = 100, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    scores = cross_val_score(rfr_time, X[s], y[s]['mins_to_sector_leave'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))


0 -> -8.217055
1 -> -6.154367
2 -> -7.456639


In [38]:
for s in sectors:
    rfr_pos = RandomForestRegressor(n_estimators = 100, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    scores = cross_val_score(rfr_pos, X[s], y[s]['sector_leave_lat'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))


0 -> -0.024419
1 -> -0.060614
2 -> -0.003453


In [39]:
for s in sectors:
    rfr_sog = RandomForestRegressor(n_estimators = 100, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    scores = cross_val_score(rfr_cog, X[s], y[s]['sector_leave_sog'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))


0 -> -1.131956
1 -> -0.999668
2 -> -1.303380


In [32]:

for s in sectors:
    rfr_cog = RandomForestRegressor(n_estimators = 40, oob_score= 'true', max_depth = 30,  random_state = 3, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    scores = cross_val_score(rfr_cog, X[s], y[s]['sector_leave_cog'], cv = cv_splits[s], scoring = 'neg_mean_absolute_error')
    print("%d -> %f" % (s, scores.mean()))


0 -> -4.991198
1 -> -3.154776
2 -> -3.743138


In [33]:
rfr_time_list = []
for s in sectors:
    rfr_time = RandomForestRegressor(n_estimators = 100, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    rfr_time.fit(X_train[s], y_train[s]['mins_to_sector_leave'])
    rfr_time_list.append(rfr_time)

In [37]:
rfr_pos_list = []
for s in sectors:
    rfr_pos = RandomForestRegressor(n_estimators = 50, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    rfr_pos.fit(X_train[s], y_train[s]['sector_leave_lat'])
    rfr_pos_list.append(rfr_pos)

In [36]:
rfr_sog_list = []
for s in sectors:
    rfr_sog = RandomForestRegressor(n_estimators = 50, oob_score= 'true', max_depth = 40,  random_state = 2, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    rfr_sog.fit(X_train[s], y_train[s]['sector_leave_sog'])
    rfr_sog_list.append(rfr_sog)

In [34]:
rfr_cog_list = []
for s in sectors:
    rfr_cog = RandomForestRegressor(n_estimators = 40, oob_score= 'true', max_depth = 30,  random_state = 3, criterion = 'mse', max_features = 'sqrt', min_samples_leaf= 20, n_jobs = -1)
    rfr_cog.fit(X_train[s], y_train[s]['sector_leave_cog'])
    rfr_cog_list.append(rfr_cog)

In [43]:
# Pickle KNN models
import pickle

for s in sectors:
    
    time_model = rfr_time_list[s]
    pickle.dump(time_model, open('../App/Agents/fel_rot_s%d/model_time.pkl' % (s), 'wb'))
    cog_model = rfr_cog_list[s]
    pickle.dump(cog_model, open('../App/Agents/fel_rot_s%d/model_cog.pkl' % (s), 'wb'))
    
    
    if s == 0:
        pos_model = rfr_pos_list[s]
        pickle.dump(pos_model, open('../App/Agents/fel_rot_s%d/model_position.pkl' % (s), 'wb'))
        sog_model = rfr_sog_list[s]
        pickle.dump(sog_model, open('../App/Agents/fel_rot_s%d/model_sog.pkl' % (s), 'wb'))
    else :
        pos_model = rfr_pos_list[s]
        pickle.dump(pos_model, open('../App/Agents/fel_rot_s%d/model_position.pkl' % (s), 'wb'))
        sog_model = rfr_sog_list[s]
        pickle.dump(sog_model, open('../App/Agents/fel_rot_s%d/model_sog.pkl' % (s), 'wb'))


In [45]:
time_model = knn_cog_list[1]
pickle.dump(time_model, open('../App/Agents/fel_rot_s%d/model_cog.pkl' % (1), 'wb'))

In [ ]:
test_loaded = pickle.load(open('../App/Agents/fel_rot_s%d/model_time.pkl' % (1), 'rb'))
test_loaded.predict([[51.93, 1.3]])[0]

In [46]:
print(knn_time_list[2])
print(test_loaded)

NameError: name 'knn_time_list' is not defined